Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8-minimal
Suggested packages:
  python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8 python3.8-minimal
0 upgraded, 4 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,676 kB of archives.
After this operation, 18.5 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.8-minimal amd64 3.8.13-1+bionic1 [762 kB]
0% [1 libpython3.8-minimal 2,624 B/762 kB 0%]^C


Python 3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
KeyboardInterrupt
>>> 
KeyboardInterrupt
>>> ^C


In [ ]:
import nibabel as nib
import torch
from model.ResNet import resnet18
from torch.autograd import Variable
import cv2
import numpy as np
from scipy import ndimage as nd

# # create main window
# root = tkinter.Tk()
# root.title('COVID-19 AI Diagnostic system')

# # window config
# root.geometry('800x600+100+50')  # width x height + widthoffset + heightoffset
# root.configure(bg='white')
# root.resizable(False, False)
# root.focusmodel()

# # logo
# img = ImageTk.PhotoImage(Image.open('img.png'))
# label = Label(image=img, border=False)
# label.place(x=530, y=0)

# ct
ct_img = None

# load model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net = resnet18(pretrained=False, b_RGB=False)
net = torch.load('best_ckpt_cnn_33_0.7851.pth',map_location=torch.device(device))#.module.state_dict())
# print(net)
net = net.cuda()
net.eval()
ct_img = nib.load("study_0256.nii.gz").get_fdata()


# def clear():
#     output_valueText.delete(1.0, 'end')

# def exit():
#     root.destroy()
def concatenate(list):
    head = list[0]

    for i in range(1, len(list)):
        head = np.concatenate((head, list[i]), 0)

    return np.array(head)

    """
Downsample the image and unify the number of CT slices
"""
def resize(img, stand_size, stand_slices):
    # img = nd.rotate(img, 90, reshape=False) # unnecessary to rotate images
    scale = float(stand_size / img.shape[0])
    slices = img.shape[2]
    if (stand_slices-slices)==1:
        return nd.zoom(input = img[:,:,:slices-1], zoom=(scale, scale, 1), order=1)
    # elif (slices-stand_slices)==1:
    #     img = nd.zoom(input = img, zoom=(scale, scale, 1), order=1)
    #     last = img[:,:,slices-1]
    #     img.append(last)
    #     return img
    return nd.zoom(input = img, zoom=(scale, scale, float(stand_slices/slices)), order=1)

"""
    resize image
"""
def process_image(img, min_side):
    size = img.shape
    h, w = size[0], size[1]
    # rescale the minside
    scale = max(w, h) / float(min_side)
    new_w, new_h = int(w/scale), int(h/scale)
    resize_img = cv2.resize(img, (new_w, new_h))
    # minside * minside
    if new_w % 2 != 0 and new_h % 2 == 0:
        top, bottom, left, right = (min_side-new_h)/2, (min_side-new_h)/2, (min_side-new_w)/2 + 1, (min_side-new_w)/2
    elif new_h % 2 != 0 and new_w % 2 == 0:
        top, bottom, left, right = (min_side-new_h)/2 + 1, (min_side-new_h)/2, (min_side-new_w)/2, (min_side-new_w)/2
    elif new_h % 2 == 0 and new_w % 2 == 0:
        top, bottom, left, right = (min_side-new_h)/2, (min_side-new_h)/2, (min_side-new_w)/2, (min_side-new_w)/2
    else:
        top, bottom, left, right = (min_side-new_h)/2 + 1, (min_side-new_h)/2, (min_side-new_w)/2 + 1, (min_side-new_w)/2
    pad_img = cv2.copyMakeBorder(resize_img, int(top), int(bottom), int(left), int(right), cv2.BORDER_CONSTANT, value=[0,0,0]) #
    return pad_img


"""
Use model to diagnose
"""
def diagnose():
    global ct_img
    ct_img = resize(ct_img,64,30)
    # clear()
    slices = ct_img.shape[2]
    diag = np.zeros(4)

            # images = [[]]
            # img_list = []
            # for id in range(tmp.shape[2]):
            #     # image size control
            #     tmp_img = process_image(tmp[:, :, id], 64)
            #     img_list.append(tmp_img)
            # images[i].append(np.array(img_list))

    img_list = []
    images = [[]]
    for i in range(30):
        img_list.append(process_image(ct_img[i],64))
    images[0].append(np.array(img_list))#.swapaxes(1,2))

    images[0] = concatenate(images[0])
    np.random.shuffle(images[0])
    print(len(images[0][1]))


    img_set = images[0][:,np.newaxis].astype(np.float32)
    print("Shape of data: " + str(img_set.shape))
    loader = torch.utils.data.DataLoader(img_set, batch_size=4)
    output = []
    for i, (img_data) in enumerate(loader):
        img_data = img_data.cuda()
        res = net(img_data)

        output.extend(np.argmax(res.detach().cpu().numpy(), 1))
        # all_preds = 

    # output = torch.tensor(np.array(output))
    print(output)
    # _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(output)
    for i in range(len(preds)):
        if preds[i] == 0:
            diag[0]+=1
        elif preds[i] == 1:
            diag[1]+=1
        elif preds[i] == 2:
            diag[2]+=1
        elif preds[i] == 3:
            diag[3]+=1
    print(diag)


diagnose()

cuda:0
64
Shape of data: (30, 1, 64, 64)
[0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 3, 3, 0]
[21.  0.  3.  6.]
